<a href="https://colab.research.google.com/github/jwyangyin/TFM/blob/main/Sistema_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="
  background:#f6f8fa;
  padding:30px;
  border:1px solid #d0d7de;
  border-radius:10px;
">
  <b style="font-size:18px;">Índice</b>
  <ul>
    <li><a href="#intro">1. Introducción</a></li>
    <li><a href="#fuentes">1.1. Fuentes de datos y estructura</a></li>
    <li><a href="#carga">1.2. Carga de datos y contexto</a></li>
    <li><a href="#estadisticas">1.4. Estadísticas básicas del subconjunto All_Beauty</a></li>
    <li><a href="#enfoque">2. Enfoque del sistema de recomendación híbrido</a></li>
    <li><a href="#siguientes">Siguientes pasos (Bloques posteriores)</a></li>
  </ul>
</div>

<a id="intro"></a>
<h1>1. Introducción</h1>

<p>
En este cuaderno implementamos un <b>sistema de recomendación híbrido</b> para el subconjunto
<b>All_Beauty</b> del dataset de reseñas de Amazon. Este enfoque combina:
</p>

<ul>
  <li><b>Filtrado colaborativo Item-to-Item</b> (Notebook 1): explota patrones de co-interacción entre usuarios e ítems.</li>
  <li><b>Recomendación basada en contenido</b> (Notebook 2): explota similitud textual/semántica entre productos usando metadatos.</li>
</ul>

<p>
La motivación del enfoque híbrido es reducir limitaciones típicas de cada estrategia:
</p>

<ul>
  <li><b>Item-to-Item</b> suele sufrir en escenarios de <b>alta esparsidad</b> y <b>cold-start de ítems</b>.</li>
  <li><b>Content-Based</b> puede generar recomendaciones <b>poco diversas</b> o “encerradas” en el perfil textual.</li>
</ul>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Conclusión del apartado:</b> el recomendador híbrido busca aprovechar la fortaleza del patrón colaborativo
  cuando existe señal suficiente, y complementarlo con contenido para mejorar cobertura, robustez y cold-start.
</div>

<a id="fuentes"></a>
<h2>1.1. Fuentes de datos y estructura</h2>

<p>
Utilizamos los mismos dos ficheros ya empleados en los cuadernos previos:
</p>

<ul>
  <li><b>Dataset A — User Reviews</b>: interacciones usuario–producto (reseñas) en formato JSONL.</li>
  <li><b>Dataset B — Item Metadata</b>: metadatos del producto (título, descripción, categorías, etc.) en formato JSONL.</li>
</ul>

<p>
Campos principales (resumen):
</p>

<ul>
  <li><b>User Reviews</b>: <code>user_id</code>, <code>parent_asin</code>, <code>rating</code>, <code>timestamp</code>…</li>
  <li><b>Item Metadata</b>: <code>parent_asin</code>, <code>title</code>, <code>description</code>, <code>features</code>, <code>categories</code>…</li>
</ul>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Conclusión del apartado:</b> el sistema híbrido requiere <b>señal colaborativa</b> (interacciones) y
  <b>representación de contenido</b> (documento textual por ítem), por lo que ambos datasets son imprescindibles.
</div>

<a id="carga"></a>
<h2>1.2. Carga de datos</h2>

<p>
La carga de datos replica la lógica usada en los Notebooks 1 y 2. Asumimos que los JSONL
están accesibles desde Google Drive (o localmente en Colab). Por lo tanto, se mantiene consistencia con los cuadernos previos para garantizar
  reproducibilidad y comparación posterior (full vs muestra).
</p>

</div>

In [1]:
# Montamos nuestro Google Drive:
from google.colab import drive
drive.mount('/content/drive')

# Importamos las librerías necesarias:
import json
import pandas as pd

# Definimos las rutas donde están ambos archivos en formato JSONL subidos a Google Colab:
path_reviews = '/content/drive/My Drive/Colab Notebooks/All_Beauty.jsonl'
path_meta    = '/content/drive/My Drive/Colab Notebooks/meta_All_Beauty.jsonl'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Cargamos el dataset de reseñas (User Reviews):
reviews = []
with open(path_reviews, "r", encoding="utf-8") as f:
    for line in f:
        reviews.append(json.loads(line.strip()))
df_reviews = pd.DataFrame(reviews)

# Mostramos las primeras líneas del dataset de reseña:
df_reviews.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Such a lovely scent but not overpowering.,This spray is really nice. It smells really go...,[],B00YQ6X8EO,B00YQ6X8EO,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1588687728923,0,True
1,4.0,Works great but smells a little weird.,"This product does what I need it to do, I just...",[],B081TJ8YS3,B081TJ8YS3,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1588615855070,1,True
2,5.0,Yes!,"Smells good, feels great!",[],B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,1589665266052,2,True
3,1.0,Synthetic feeling,Felt synthetic,[],B09JS339BZ,B09JS339BZ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,1643393630220,0,True
4,5.0,A+,Love it,[],B08BZ63GMJ,B08BZ63GMJ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,1609322563534,0,True


In [3]:
# Cargamos el dataset de metadatos (Item Metadata):
meta = []
with open(path_meta, "r", encoding="utf-8") as f:
    for line in f:
        meta.append(json.loads(line.strip()))
df_meta = pd.DataFrame(meta)

# Mostramos las primeras líneas del dataset de metadatos:
df_meta.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together
0,All Beauty,"Howard LC0008 Leather Conditioner, 8-Ounce (4-...",4.8,10,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Howard Products,[],{'Package Dimensions': '7.1 x 5.5 x 3 inches; ...,B01CUPMQZE,None
1,All Beauty,Yes to Tomatoes Detoxifying Charcoal Cleanser ...,4.5,3,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Yes To,[],"{'Item Form': 'Powder', 'Skin Type': 'Acne Pro...",B076WQZGPM,None
2,All Beauty,Eye Patch Black Adult with Tie Band (6 Per Pack),4.4,26,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Levine Health Products,[],{'Manufacturer': 'Levine Health Products'},B000B658RI,None
3,All Beauty,"Tattoo Eyebrow Stickers, Waterproof Eyebrow, 4...",3.1,102,[],[],NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Cherioll,[],"{'Brand': 'Cherioll', 'Item Form': 'Powder', '...",B088FKY3VD,None
4,All Beauty,Precision Plunger Bars for Cartridge Grips – 9...,4.3,7,"[Material: 304 Stainless Steel; Brass tip, Len...",[The Precision Plunger Bars are designed to wo...,NaN,[{'thumb': 'https://m.media-amazon.com/images/...,[],Precision,[],{'UPC': '644287689178'},B07NGFDN6G,None


In [4]:
# Mostramos las dimensiones ambos datasets referidos:
print("Las dimensiones de cada dataset son las siguientes:")
print("df_reviews shape:", df_reviews.shape)
print("df_meta shape:", df_meta.shape)

Las dimensiones de cada dataset son las siguientes:
df_reviews shape: (701528, 10)
df_meta shape: (112590, 14)


<a id="contexto"></a>
<h2>1.3. Contexto de los datos y definición del objetivo</h2>

<p>
El subconjunto <b>All_Beauty</b> contiene interacciones usuario–producto a partir de reseñas de Amazon.
Cada registro de reseña representa una evidencia explícita (p.ej. <code>rating</code>) y un evento temporal
(p.ej. <code>timestamp</code>), mientras que el dataset de metadatos aporta información descriptiva del producto
(p.ej. <code>title</code>, <code>description</code>, <code>features</code>, <code>categories</code>).
</p>

<p>
En este TFM, el objetivo es construir un recomendador que, dado un usuario, sea capaz de sugerir un top-N de
productos relevantes del dominio <b>belleza</b>. Para ello, se consideran dos fuentes de señal:
</p>

<ul>
  <li>
    <b>Señal colaborativa</b>: patrones de consumo/reseña compartidos entre usuarios y productos (Notebook 1).
  </li>
  <li>
    <b>Señal de contenido</b>: similitud semántica/textual entre productos usando metadatos (Notebook 2).
  </li>
</ul>

<p>
<b>Decisiones de modelado (alineadas con los cuadernos previos):</b>
</p>

<ul>
  <li>
    Se transforma la señal explícita (<code>rating</code>) a una señal implícita cuando sea necesario
    (por ejemplo, considerar interacción positiva si <code>rating ≥ 4</code>).
  </li>
  <li>
    La unidad de ítem será <code>parent_asin</code> para permitir el cruce consistente entre reseñas y metadatos.
  </li>
  <li>
    Se aplican filtros de consistencia (p.ej. eliminar registros sin <code>user_id</code> o <code>parent_asin</code>,
    y manejar valores nulos en campos textuales del contenido).
  </li>
</ul>

<p>
Finalmente, el <b>sistema híbrido</b> combinará las predicciones de ambos enfoques a nivel de score, buscando
mejorar:
</p>

<ul>
  <li><b>Calidad</b>: precisión y ranking (Precision@K, Recall@K, NDCG/MAP según se use).</li>
  <li><b>Cobertura</b>: proporción de catálogo que puede recomendarse.</li>
  <li><b>Robustez</b>: mitigación de cold-start parcial y esparsidad del colaborativo.</li>
</ul>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Conclusión del apartado:</b> el problema se plantea como recomendación Top-N en All_Beauty,
  integrando señal colaborativa (interacciones) y señal de contenido (metadatos) para mejorar calidad,
  cobertura y robustez frente a esparsidad y cold-start.
</div>

<a id="estadisticas"></a>
<h2>1.4. Estadísticas básicas del subconjunto All_Beauty</h2>

<p>
Se calculan métricas descriptivas mínimas para contextualizar el problema:
</p>

<ul>
  <li>Número de interacciones (reseñas)</li>
  <li>Número de usuarios únicos</li>
  <li>Número de ítems únicos (<code>parent_asin</code>)</li>
</ul>

<p>
(El diagnóstico profundo de esparsidad, long tail y calidad del texto se desarrolló en los Notebooks 1 y 2,
y aquí se reutiliza como referencia sin duplicarlo.)
</p>

In [5]:
# ============================================================
# Estadísticas básicas (All_Beauty)
# ============================================================

#Creamos las variables necesarias:
n_reviews = df_reviews.shape[0]
n_users   = df_reviews["user_id"].nunique()
n_items_r = df_reviews["parent_asin"].nunique()

n_items_m = df_meta["parent_asin"].nunique()
n_rows_m  = df_meta.shape[0]

# Imprimimos los resultados obtenidos:
print("\nEstadísticas básicas (All_Beauty - User Reviews):")
print(f"- Reseñas (interacciones): {n_reviews:,}".replace(",", "."))
print(f"- Usuarios únicos: {n_users:,}".replace(",", "."))
print(f"- Ítems únicos (en reseñas): {n_items_r:,}".replace(",", "."))

print("\nEstadísticas básicas (All_Beauty - Item Metadata):")
print(f"- Filas (productos): {n_rows_m:,}".replace(",", "."))
print(f"- parent_asin únicos: {n_items_m:,}".replace(",", "."))


Estadísticas básicas (All_Beauty - User Reviews):
- Reseñas (interacciones): 701.528
- Usuarios únicos: 631.986
- Ítems únicos (en reseñas): 112.565

Estadísticas básicas (All_Beauty - Item Metadata):
- Filas (productos): 112.590
- parent_asin únicos: 112.590


<hr>

<a id="modelo"></a>
<h1>2. Enfoque del modelo: Sistema de Recomendación Híbrido</h1>

<p>
En este capítulo se describe el enfoque conceptual del <b>sistema de recomendación híbrido</b> propuesto en este TFM.
Este sistema combina dos estrategias ya desarrolladas previamente:
</p>

<ul>
  <li><b>Filtrado Colaborativo Item-to-Item</b></li>
  <li><b>Recomendación Basada en Contenido</b></li>
</ul>

<p>
La motivación principal del enfoque híbrido es aprovechar la <b>complementariedad</b> entre ambos modelos,
integrando información procedente del comportamiento colectivo de los usuarios y de las características
intrínsecas de los productos.
</p>

<a id="intuicion"></a>
<h2>2.1. Intuición del enfoque híbrido</h2>

<p>
La intuición del sistema híbrido parte de una observación sencilla:
<b>los productos pueden ser relevantes para un usuario por distintos motivos</b>.
</p>

<ul>
  <li>
    Dos productos pueden ser relevantes porque <b>otros usuarios con patrones similares los han consumido conjuntamente</b>
    (señal colaborativa).
  </li>
  <li>
    Dos productos pueden ser relevantes porque <b>comparten características semánticas o funcionales similares</b>
    (señal de contenido).
  </li>
</ul>

<p>
El sistema híbrido integra ambas perspectivas para generar recomendaciones más robustas y equilibradas.
</p>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Regla práctica:</b> “recomendamos productos que son similares a los ya consumidos por el usuario,
  tanto por patrones de comportamiento global como por similitud de contenido”.
</div>

<a id="representacion"></a>
<h2>2.2. Representación de los datos</h2>

<p>
El sistema híbrido utiliza una representación dual de los datos:
</p>

<ul>
  <li>
    <b>Matriz usuario–ítem</b> para modelar interacciones y calcular similitud Item-to-Item.
  </li>
  <li>
    <b>Representación vectorial de ítems</b> basada en contenido textual (TF-IDF).
  </li>
</ul>

<p>
En ambos casos, los ítems están identificados de forma consistente mediante el campo
<code>parent_asin</code>, lo que permite integrar ambas fuentes de información.
</p>

<p>
La señal de preferencia del usuario puede definirse como:
</p>

<ul>
  <li>
    <b>Explícita:</b> utilizando directamente el <code>rating</code>.
  </li>
  <li>
    <b>Implícita:</b> transformando el rating en una interacción positiva (por ejemplo, <code>rating ≥ 4</code>).
  </li>
</ul>

<a id="senales"></a>
<h2>2.3. Señales utilizadas en el sistema híbrido</h2>

<p>
El sistema híbrido integra dos tipos de señal complementarias:
</p>

<h4>Señal colaborativa</h4>
<ul>
  <li>Interacciones usuario–producto extraídas de las reseñas.</li>
  <li>Co-ocurrencias de productos en historiales de usuarios.</li>
  <li>Similitud Item-to-Item basada en patrones de consumo compartidos.</li>
</ul>

<h4>Señal basada en contenido</h4>
<ul>
  <li>Texto descriptivo de los productos (título, descripción, características).</li>
  <li>Vectorización mediante TF-IDF.</li>
  <li>Similitud semántica entre productos usando similitud del coseno.</li>
</ul>

<p>
Ambas señales se calculan de forma independiente y se combinan posteriormente en el proceso de recomendación.
</p>

<a id="hibridacion"></a>
<h2>2.4. Estrategia de combinación (hibridación)</h2>

<p>
La estrategia adoptada es una <b>hibridación a nivel de score</b>.
Cada modelo genera un score de relevancia para un conjunto de ítems candidatos, y dichos scores se combinan
para obtener una puntuación final.
</p>

<p>
Formalmente, para un usuario <code>u</code> y un ítem candidato <code>i</code>:
</p>

<pre style="background:#f6f8fa;border:1px solid #d0d7de;padding:10px;border-radius:8px;">
score_híbrido(i | u) =
  α · score_colaborativo_norm(i | u) +
  (1 − α) · score_contenido_norm(i | u)
</pre>

<p>
donde el parámetro <b>α</b> controla el peso relativo de la señal colaborativa frente a la señal de contenido.
</p>

<p>
Esta estrategia permite ajustar el comportamiento del sistema en función de las características del dominio
y del nivel de información disponible.
</p>

<a id="recomendaciones"></a>
<h2>2.5. Generación de recomendaciones</h2>

<p>
El proceso de recomendación para un usuario sigue los siguientes pasos:
</p>

<ol>
  <li>
    Se identifican los ítems con los que el usuario ha interactuado positivamente.
  </li>
  <li>
    Para cada uno de esos ítems, se obtienen:
    <ul>
      <li>Ítems similares según el modelo Item-to-Item.</li>
      <li>Ítems similares según el modelo basado en contenido.</li>
    </ul>
  </li>
  <li>
    Se agregan los ítems candidatos, combinando los scores de ambos modelos.
  </li>
  <li>
    Se eliminan los ítems ya consumidos por el usuario y se genera un ranking Top-N final.
  </li>
</ol>

<p>
El resultado es una lista de recomendaciones que integra patrones de comportamiento colectivo
y similitud semántica entre productos.
</p>

<a id="justificacion"></a>
<h2>2.6. Justificación del enfoque híbrido</h2>

<p>
El enfoque híbrido es especialmente adecuado para el dataset All_Beauty debido a:
</p>

<ul>
  <li>Alta esparsidad de la matriz usuario–ítem.</li>
  <li>Existencia de productos con pocas interacciones.</li>
  <li>Disponibilidad de metadatos ricos en contenido textual.</li>
</ul>

<p>
Además, el sistema híbrido permite mejorar:
</p>

<ul>
  <li><b>Cobertura</b> del catálogo.</li>
  <li><b>Robustez</b> frente a cold-start parcial.</li>
  <li><b>Calidad</b> del ranking de recomendaciones.</li>
</ul>

<a id="limitaciones"></a>
<h2>2.7. Limitaciones del enfoque híbrido</h2>

<p>
A pesar de sus ventajas, el sistema híbrido presenta algunas limitaciones:
</p>

<ul>
  <li>
    <b>Mayor complejidad computacional</b> al combinar dos modelos.
  </li>
  <li>
    <b>Necesidad de calibración</b> del parámetro α.
  </li>
  <li>
    <b>Dependencia de la calidad del contenido textual</b> disponible.
  </li>
</ul>

<p>
En el siguiente capítulo se desarrollará el análisis exploratorio específico y la preparación de los datos
necesaria para implementar este sistema híbrido.
</p>

<hr>

<a id="cap3"></a>
<h1>3. Análisis exploratorio y diagnóstico del dataset para el sistema híbrido</h1>

<p>
En este capítulo se realiza un análisis exploratorio y diagnóstico <b>específico</b> orientado al diseño
del sistema de recomendación híbrido. No se repite el análisis exploratorio exhaustivo ya realizado en
los sistemas Item-to-Item y Basado en Contenido, sino que se revisan únicamente aquellos aspectos
críticos para garantizar la correcta integración de ambos enfoques.
</p>

<p>
El objetivo principal es validar que el dataset proporciona:
</p>

<ul>
  <li>Señal colaborativa suficiente para el filtrado Item-to-Item.</li>
  <li>Información de contenido adecuada para representar los ítems.</li>
  <li>Consistencia entre ambos conjuntos de datos.</li>
</ul>

<a id="consistencia"></a>
<h2>3.1. Consistencia entre interacciones y metadatos</h2>

<p>
Un requisito fundamental del sistema híbrido es que los ítems presentes en las interacciones
(usuario–producto) puedan enlazarse correctamente con sus metadatos de contenido.
</p>

<p>
Para ello, se verifica:
</p>

<ul>
  <li>El grado de solapamiento de <code>parent_asin</code> entre <b>df_reviews</b> y <b>df_meta</b>.</li>
  <li>La existencia de ítems con interacciones pero sin información de contenido.</li>
</ul>

In [6]:
# ============================================================
# Consistencia entre interacciones y metadatos
# ============================================================

items_reviews = set(df_reviews["parent_asin"].unique())
items_meta    = set(df_meta["parent_asin"].unique())

common_items  = items_reviews.intersection(items_meta)
only_reviews  = items_reviews - items_meta
only_meta     = items_meta - items_reviews

print(f"Ítems en reseñas: {len(items_reviews):,}".replace(",", "."))
print(f"Ítems en metadatos: {len(items_meta):,}".replace(",", "."))
print(f"Ítems comunes: {len(common_items):,}".replace(",", "."))
print(f"Ítems solo en reseñas: {len(only_reviews):,}".replace(",", "."))
print(f"Ítems solo en metadatos: {len(only_meta):,}".replace(",", "."))

Ítems en reseñas: 112.565
Ítems en metadatos: 112.590
Ítems comunes: 112.565
Ítems solo en reseñas: 0
Ítems solo en metadatos: 25


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Conclusión del apartado:</b> el análisis muestra un solapamiento completo entre los ítems con
  interacciones y los metadatos disponibles: todos los productos presentes en las reseñas cuentan
  con información de contenido asociada. Adicionalmente, se identifican un pequeño número de ítems
  con metadatos pero sin interacciones, que constituyen un escenario natural de <i>cold-start de ítems</i>,
  donde el componente basado en contenido del sistema híbrido resulta especialmente relevante.
</div>

<h2>3.2. Diagnóstico de la señal colaborativa</h2>

<p>
El componente colaborativo del sistema híbrido se apoya en las interacciones
usuario–producto extraídas de las reseñas. Tal como se desarrolló en el Notebook 1,
estas interacciones permiten construir una matriz usuario–ítem de gran tamaño,
sobre la que se calculan relaciones de similitud Item-to-Item.
</p>

<p>
El análisis exploratorio realizado previamente pone de manifiesto varias características
clave del dataset:
</p>

<ul>
  <li>
    La matriz usuario–ítem presenta una <b>alta esparsidad</b>, con un gran número de
    usuarios que cuentan con historiales de interacción cortos.
  </li>
  <li>
    Existe una distribución <b>long-tail</b> en el número de interacciones por producto,
    donde una pequeña fracción de ítems concentra la mayor parte de las reseñas.
  </li>
  <li>
    A pesar de esta esparsidad, el volumen total de interacciones es suficiente para
    estimar <b>relaciones Item-to-Item estables</b> para una parte significativa del catálogo.
  </li>
</ul>

<p>
Estas propiedades hacen que el filtrado colaborativo Item-to-Item sea especialmente
adecuado en escenarios donde el usuario ha interactuado al menos con uno o varios
productos, ya que permite generar recomendaciones sin necesidad de construir un perfil
explícito de usuario.
</p>

<p>
No obstante, la presencia de productos con pocas o ninguna interacción limita la capacidad
del modelo colaborativo en escenarios de <i>cold-start de ítems</i>, donde no es posible
inferir relaciones fiables únicamente a partir del comportamiento histórico.
</p>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Conclusión del apartado:</b> la señal colaborativa resulta adecuada para capturar
  relaciones Item-to-Item relevantes en una parte sustancial del catálogo, pero su
  naturaleza dispersa y la existencia de ítems con baja interacción justifican su
  combinación con un modelo basado en contenido dentro de un sistema híbrido.
</div>

<h2>3.3. Diagnóstico de la señal de contenido</h2>

<p>
El componente basado en contenido del sistema híbrido depende de la disponibilidad
y calidad de la información descriptiva asociada a cada producto. En el Notebook 2
se analizó la <b>cobertura real de contenido</b>, entendida como la proporción de productos
que disponen de información no vacía y semánticamente utilizable.
</p>

<p>
El análisis de los campos textuales muestra que:
</p>

<ul>
  <li>
    El campo <code>title</code> presenta una cobertura prácticamente completa y constituye
    la principal fuente de información semántica textual del catálogo.
  </li>
  <li>
    Los campos <code>description</code> y <code>features</code> presentan una cobertura
    considerablemente menor, por lo que su contribución semántica es parcial y no homogénea
    en el conjunto de productos.
  </li>
</ul>

<p>
De forma complementaria, el análisis de atributos estructurados (no textuales) revela que
campos como <code>main_category</code>, <code>details</code> y <code>store</code> presentan
una cobertura elevada, lo que permite incorporar información contextual adicional a la
representación de los ítems.
</p>

<p>
En el contexto del sistema híbrido, la señal de contenido desempeña un doble rol fundamental:
</p>

<ul>
  <li>
    Complementar la señal colaborativa en escenarios de alta esparsidad o baja interacción.
  </li>
  <li>
    Permitir la recomendación de productos nuevos o poco populares, donde el filtrado
    colaborativo no dispone de información suficiente (<i>cold-start de ítems</i>).
  </li>
</ul>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Conclusión del apartado:</b> la disponibilidad de información de contenido, especialmente
  a través del campo <code>title</code> y de determinados atributos estructurados con alta
  cobertura, hace viable y justificada la incorporación de un modelo basado en contenido
  como componente del sistema de recomendación híbrido.
</div>

<h2>3.4. Implicaciones del diagnóstico para el sistema de recomendación híbrido</h2>

<p>
El análisis realizado en los apartados anteriores permite extraer una serie de implicaciones
directas para el diseño y la implementación del sistema de recomendación híbrido.
Estas implicaciones derivan tanto de las características de la señal colaborativa como
de la disponibilidad y calidad de la señal de contenido.
</p>

<p>
En primer lugar, el diagnóstico de la señal colaborativa muestra que, aunque el volumen
global de interacciones es suficiente para aprender relaciones Item-to-Item relevantes,
la alta esparsidad de la matriz usuario–ítem y la presencia de productos con pocas
interacciones limitan la capacidad del filtrado colaborativo en determinados escenarios.
</p>

<p>
En segundo lugar, el análisis de la señal de contenido evidencia que existe información
descriptiva suficiente para representar semánticamente los productos, si bien dicha
información no es homogénea en todos los campos. En particular, el campo <code>title</code>
y determinados atributos estructurados presentan una cobertura elevada, mientras que
otros campos textuales aportan información solo de forma parcial.
</p>

<p>
A partir de estos resultados, se derivan las siguientes decisiones de diseño para el
sistema híbrido:
</p>

<ul>
  <li>
    Combinar la señal colaborativa y la señal de contenido a nivel de score, de forma que
    ambas contribuyan de manera complementaria a la recomendación final.
  </li>
  <li>
    Priorizar los campos de contenido con alta cobertura y relevancia semántica en la
    representación de los ítems.
  </li>
  <li>
    Incorporar mecanismos de normalización y ponderación que permitan equilibrar la
    contribución de cada componente del sistema.
  </li>
  <li>
    Diseñar el sistema de forma que pueda generar recomendaciones tanto para usuarios
    con historial previo como para escenarios de baja interacción o <i>cold-start</i> de ítems.
  </li>
</ul>

<p>
Estas decisiones orientan directamente la fase de preparación de datos y construcción
del modelo híbrido, que se desarrollan en el siguiente capítulo.
</p>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Conclusión del apartado:</b> el diagnóstico confirma que ninguna de las señales disponibles
  es suficiente por sí sola para cubrir todos los escenarios del dominio All_Beauty. La
  combinación de señal colaborativa y de contenido emerge como una solución justificada
  y necesaria, y define los requisitos técnicos que guían la preparación de datos y la
  implementación del sistema de recomendación híbrido.
</div>

<hr>

<a id="cap4"></a>
<h1>4. Preparación de datos para el sistema de recomendación híbrido</h1>

<p>
En este capítulo se describen los pasos de preparación de datos necesarios para la
construcción del sistema de recomendación híbrido. El objetivo es definir un pipeline
coherente que permita integrar de forma consistente el componente colaborativo
(Item-to-Item) y el componente basado en contenido.
</p>

<p>
La preparación de datos sigue los mismos principios metodológicos adoptados en los
Notebooks 1 y 2, garantizando la coherencia entre modelos y permitiendo comparaciones
justas en fases posteriores de evaluación.
</p>

<p>
Los pasos descritos a continuación se aplican inicialmente sobre el dataset completo.
Más adelante, se reutilizará este mismo pipeline sobre una muestra del dataset para
analizar el impacto del tamaño de los datos en el rendimiento del sistema híbrido.
</p>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Conclusión del apartado:</b> este capítulo establece un pipeline de preparación común
  que asegura la coherencia metodológica entre los distintos sistemas de recomendación
  desarrollados en este trabajo.
</div>

<h2>4.1. Definición de la señal de interacción</h2>

<p>
El sistema de recomendación híbrido se basa en las interacciones usuario–producto
extraídas de las reseñas. Aunque el dataset proporciona valoraciones explícitas
(<code>rating</code>), en este trabajo se adopta un enfoque de <b>feedback implícito</b>,
siguiendo la misma estrategia utilizada en el sistema Item-to-Item.
</p>

<p>
En concreto, se considera que una interacción es positiva cuando la valoración del
usuario es igual o superior a un umbral predefinido:
</p>

<ul>
  <li><b>Interacción positiva:</b> <code>rating ≥ 4</code></li>
</ul>

<p>
Este criterio permite simplificar la representación de las interacciones y resulta
adecuado para sistemas de recomendación orientados a ranking Top-N. Además, es
compatible tanto con el filtrado colaborativo como con el enfoque basado en contenido,
que pueden operar a partir de una única interacción positiva.
</p>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Conclusión del apartado:</b> la transformación de la señal explícita en feedback
  implícito permite unificar el tratamiento de las interacciones y mantener coherencia
  con los sistemas de recomendación desarrollados previamente.
</div>

In [7]:
# ============================================================
# 4.1 Definición de la señal implícita
# ============================================================

df_interactions = df_reviews.copy()

RATING_THRESHOLD = 4  # interacción positiva si rating >= 4

df_interactions["interaction"] = (
    df_interactions["rating"] >= RATING_THRESHOLD
).astype(int)

# Nos quedamos solo con interacciones positivas
df_interactions = df_interactions[df_interactions["interaction"] == 1]

df_interactions = df_interactions[["user_id", "parent_asin", "interaction"]]

print("Interacciones positivas:", df_interactions.shape[0])
print("Usuarios únicos:", df_interactions["user_id"].nunique())
print("Ítems únicos:", df_interactions["parent_asin"].nunique())

display(df_interactions.head())

Interacciones positivas: 500107
Usuarios únicos: 455586
Ítems únicos: 91187


,user_id,parent_asin,interaction
0,AGKHLEW2SOWHNMFQIJGBECAF7INQ,B00YQ6X8EO,1
1,AGKHLEW2SOWHNMFQIJGBECAF7INQ,B081TJ8YS3,1
2,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,B097R46CSY,1
4,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,B08BZ63GMJ,1
5,AGMJ3EMDVL6OWBJF7CA5RGJLXN5A,B00R8DXL44,1


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
  <b>Aclaración metodológica:</b> las métricas obtenidas en este apartado (4.1) describen la señal implícita
  antes de construir la matriz usuario–ítem y antes de aplicar restricciones del componente colaborativo
  (p. ej., soporte mínimo por ítem). Por ello, los recuentos pueden diferir de los reportados en el Notebook 1
  en secciones posteriores donde ya se trabaja sobre la matriz CSR y se filtran ítems con baja evidencia
  colaborativa (por ejemplo, <code>nnz ≥ 2</code>).
</div>

<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado:</b> la transformación de la señal explícita en feedback implícito
mediante el umbral <code>rating ≥ 4</code> permite unificar el tratamiento de las interacciones
y mantener coherencia con el sistema Item-to-Item desarrollado previamente. Los resultados
obtenidos muestran que este criterio conserva un volumen elevado de interacciones positivas,
usuarios e ítems, proporcionando una base suficientemente rica para el aprendizaje de
patrones de recomendación orientados a ranking Top-N y adecuada para la integración del
componente colaborativo y el componente basado en contenido.
</div>

<h2>4.2. Filtrado básico de consistencia</h2>

<p>
Antes de aplicar cualquier filtrado por frecuencia o construir representaciones para
el sistema híbrido, se realiza un filtrado básico de consistencia. Este paso garantiza
la integridad estructural de los datos y evita errores posteriores en la construcción
de matrices y en el cálculo de similitudes.
</p>

<p>
En concreto, se aplican los siguientes criterios:
</p>

<ul>
  <li>Eliminación de registros sin identificador de usuario o de producto.</li>
  <li>Conservación únicamente de ítems para los que existe información de metadatos.</li>
</ul>

<p>
Este filtrado es conservador y no pretende reducir el volumen de datos, sino asegurar
la coherencia entre las distintas fuentes utilizadas por el sistema híbrido.
</p>

In [8]:
# ============================================================
# 4.2 Filtrado básico de consistencia
# ============================================================

# Eliminar registros incompletos
df_interactions = df_interactions.dropna(subset=["user_id", "parent_asin"])

# Mantener solo ítems con metadatos (coherente con Notebook 2)
valid_items = set(df_meta["parent_asin"].unique())
df_interactions = df_interactions[
    df_interactions["parent_asin"].isin(valid_items)
]

print("Tras filtrado de consistencia:")
print("Interacciones:", df_interactions.shape[0])
print("Usuarios únicos:", df_interactions["user_id"].nunique())
print("Ítems únicos:", df_interactions["parent_asin"].nunique())

Tras filtrado de consistencia:
Interacciones: 500107
Usuarios únicos: 455586
Ítems únicos: 91187


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado:</b> el filtrado básico de consistencia confirma la alta calidad
estructural del dataset, ya que no se produce pérdida de interacciones relevantes tras la
eliminación de registros incompletos y la restricción a ítems con metadatos disponibles.
Este resultado garantiza la coherencia entre las interacciones de usuario y la información
de contenido, proporcionando una base sólida y alineada para la construcción de las
representaciones necesarias en el sistema de recomendación híbrido.
</div>

<h2>4.3. Filtrado por frecuencia para estabilidad del sistema híbrido</h2>

<p>
Tras garantizar la consistencia estructural de los datos, se aplica un filtrado por
frecuencia de interacción con el objetivo de mejorar la estabilidad del componente
colaborativo, sin comprometer la cobertura necesaria para el componente basado en
contenido.
</p>

<p>
Siguiendo el enfoque adoptado en el sistema Item-to-Item, este filtrado se diseña de
forma <b>mínima y configurable</b>. En particular, se evita la eliminación agresiva de
usuarios o ítems, ya que el sistema híbrido debe ser capaz de operar en escenarios de
historial corto y baja evidencia colaborativa.
</p>

<p>
El filtrado se aplica de manera independiente a usuarios e ítems, permitiendo analizar
su impacto y ajustar los umbrales en función del comportamiento del modelo.
</p>

<h3>4.3.1. Filtrado mínimo de usuarios</h3>

<p>
En este subapartado se filtran los usuarios en función del número de interacciones
positivas registradas. Siguiendo la misma estrategia que en el sistema Item-to-Item,
se establece un umbral bajo que permite conservar usuarios con historiales muy cortos.
</p>

<p>
Esta decisión es deliberada: tanto el filtrado colaborativo Item-to-Item como el enfoque
basado en contenido pueden generar recomendaciones a partir de una única interacción
positiva. Eliminar estos usuarios reduciría artificialmente la cobertura del sistema
híbrido y limitaría el análisis de escenarios realistas.
</p>

In [9]:
# ============================================================
# Filtrado mínimo de usuarios
# ============================================================

MIN_USER_INTERACTIONS = 1  # configurable (p.ej. 2 para pruebas)

tmp = df_interactions.copy()

user_freq = tmp.groupby("user_id")["parent_asin"].size()
valid_users = user_freq[user_freq >= MIN_USER_INTERACTIONS].index

tmp = tmp[tmp["user_id"].isin(valid_users)].copy()

# Imprimimos los resultados obtenidos:
print("Tras filtrado de usuarios:")
print("Interacciones:", tmp.shape[0])
print("Usuarios únicos:", tmp["user_id"].nunique())
print("Ítems únicos:", tmp["parent_asin"].nunique())

Tras filtrado de usuarios:
Interacciones: 500107
Usuarios únicos: 455586
Ítems únicos: 91187


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del subapartado:</b> el filtrado mínimo de usuarios con un umbral de una
interacción positiva no produce reducción en el número de usuarios ni de interacciones,
confirmando que el conjunto de datos ya cumple este criterio de forma natural. Este
resultado valida la decisión de emplear un filtrado conservador, coherente con el sistema
Item-to-Item, y permite preservar escenarios de historiales cortos en los que el
componente basado en contenido puede aportar valor adicional dentro del sistema híbrido.
</div>

<h3>4.3.2. Filtrado mínimo de ítems</h3>

<p>
En este subapartado se aplica un filtrado mínimo de ítems en función del número de
interacciones positivas registradas. Este paso tiene como objetivo reducir posibles
fuentes de ruido en el componente colaborativo, evitando ítems con evidencia
extremadamente baja para el cálculo de co-ocurrencias.
</p>

<p>
Siguiendo la misma estrategia utilizada en el sistema Item-to-Item, el umbral se
establece inicialmente en un valor bajo y configurable. De este modo, se evita eliminar
ítems potencialmente relevantes desde el punto de vista del contenido, que podrían ser
recomendados por el componente basado en contenido incluso cuando su soporte
colaborativo es limitado.
</p>

<p>
Este filtrado se concibe como un compromiso entre estabilidad colaborativa y cobertura
del catálogo, manteniendo la flexibilidad necesaria para ajustar el umbral en fases
experimentales posteriores.
</p>

In [10]:
# ============================================================
# 4.3.2 Filtrado mínimo de ítems
# ============================================================

MIN_ITEM_INTERACTIONS = 1  # configurable (p.ej. 2 o 3 en pruebas)

item_freq = tmp.groupby("parent_asin")["user_id"].size()
valid_items = item_freq[item_freq >= MIN_ITEM_INTERACTIONS].index

tmp = tmp[tmp["parent_asin"].isin(valid_items)].copy()

# Imprimimos los resultados obtenidos:
print("Tras filtrado de ítems:")
print("Interacciones:", tmp.shape[0])
print("Usuarios únicos:", tmp["user_id"].nunique())
print("Ítems únicos:", tmp["parent_asin"].nunique())

Tras filtrado de ítems:
Interacciones: 500107
Usuarios únicos: 455586
Ítems únicos: 91187


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del subapartado:</b> el filtrado mínimo de ítems con un umbral de una
interacción positiva no produce reducción en el número de ítems ni de interacciones,
lo que indica que el dataset ya satisface este criterio de forma natural. Este resultado
confirma la idoneidad de un filtrado conservador en el sistema híbrido, preservando la
cobertura del catálogo y permitiendo que ítems con bajo soporte colaborativo puedan ser
recomendados a través del componente basado en contenido.
</div>

<h3>4.3.3. Dataset resultante tras el filtrado por frecuencia</h3>

<p>
El dataset resultante tras aplicar el filtrado mínimo de usuarios e ítems mantiene el
volumen original de interacciones positivas, usuarios e ítems. Este resultado confirma
que los criterios definidos son conservadores y que el conjunto de datos ya cumple de
forma natural los requisitos mínimos de frecuencia.
</p>

<p>
A partir de este punto, el pipeline continúa con el indexado de usuarios e ítems y la
construcción de las estructuras necesarias para el cálculo de similitudes y la generación
de recomendaciones.
</p>

<a id="cap4_4"></a>
<h2>4.4. Indexado de usuarios e ítems y construcción de la matriz CSR</h2>

<p>
En este apartado se construye la representación matricial que servirá como base para el
componente colaborativo del sistema de recomendación híbrido. En concreto, se transforma
el conjunto de interacciones filtradas en una matriz usuario–ítem dispersa (CSR),
siguiendo el mismo enfoque empleado en el sistema de recomendación Item-to-Item.
</p>

<p>
Este paso es fundamental porque permite:
</p>

<ul>
  <li>Representar de forma eficiente un dataset altamente disperso.</li>
  <li>Garantizar compatibilidad directa con el cálculo de similitudes colaborativas.</li>
  <li>Establecer una estructura común sobre la que integrar posteriormente la señal de contenido.</li>
</ul>

In [12]:
# ============================================================
# Preparación del dataset base
# ============================================================

# Importamos las librerías necesarias:
from scipy.sparse import csr_matrix
import numpy as np

tmp = df_interactions.copy()

# Imprimimos los resultados obtenidos:
print("Dataset base para la matriz:")
print("Interacciones:", tmp.shape[0])
print("Usuarios únicos:", tmp["user_id"].nunique())
print("Ítems únicos:", tmp["parent_asin"].nunique())

Dataset base para la matriz:
Interacciones: 500107
Usuarios únicos: 455586
Ítems únicos: 91187


<h3>4.4.2. Indexado de usuarios e ítems</h3>

<p>
Se construyen dos diccionarios de mapeo que asignan un índice entero a cada identificador
real:
</p>

<ul>
  <li><code>user_to_idx</code>: mapea cada usuario a un índice entero.</li>
  <li><code>item_to_idx</code>: mapea cada ítem a un índice entero.</li>
</ul>

<p>
Este mapeo permite construir la matriz CSR y, posteriormente, traducir las
recomendaciones generadas al espacio original de identificadores.
</p>

In [13]:
# ============================================================
# Indexado (ID -> índice)
# ============================================================

user_ids = tmp["user_id"].astype(str).unique()
item_ids = tmp["parent_asin"].astype(str).unique()

user_to_idx = {u: i for i, u in enumerate(user_ids)}
item_to_idx = {it: j for j, it in enumerate(item_ids)}

tmp["user_idx"] = tmp["user_id"].map(user_to_idx)
tmp["item_idx"] = tmp["parent_asin"].map(item_to_idx)

n_users = len(user_ids)
n_items = len(item_ids)

print("Usuarios indexados:", n_users)
print("Ítems indexados:", n_items)

Usuarios indexados: 455586
Ítems indexados: 91187


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado:</b> el proceso de indexado de usuarios e ítems define un espacio común de representación que permite traducir de forma eficiente los identificadores reales a índices enteros, requisito indispensable para la construcción de la matriz dispersa CSR. Este paso garantiza la coherencia entre las distintas fuentes de información y proporciona una base estable sobre la que pueden operar de manera integrada el componente colaborativo y el componente basado en contenido del sistema de recomendación híbrido.
</div>

<h3>4.4.3. Construcción de la matriz usuario–ítem (CSR)</h3>

<p>
Una vez indexados usuarios e ítems, se construye la matriz dispersa <b>usuario–ítem</b>
en formato <b>CSR (Compressed Sparse Row)</b>. Esta representación es especialmente adecuada
para datasets de recomendación, donde la mayoría de combinaciones usuario–ítem no presentan
interacción (matriz altamente dispersa).
</p>

<p>
En esta matriz:
</p>

<ul>
  <li><b>Filas:</b> usuarios (<code>user_idx</code>)</li>
  <li><b>Columnas:</b> ítems (<code>item_idx</code>)</li>
  <li><b>Valores:</b> señal implícita binaria (<code>interaction = 1</code> si existe interacción positiva)</li>
</ul>

<p>
La matriz CSR permite:
</p>

<ul>
  <li>Reducir drásticamente el uso de memoria frente a una matriz densa.</li>
  <li>Acelerar operaciones posteriores de álgebra lineal y cálculo de similitudes.</li>
  <li>Escalar el sistema colaborativo a catálogos grandes y datasets dispersos.</li>
</ul>

In [14]:
# ============================================================
# Construcción de la matriz CSR (usuario x ítem)
# ============================================================

# Valores de la matriz (señal implícita binaria):
data = tmp["interaction"].astype(np.float32).values

# Índices de filas y columnas:
rows = tmp["user_idx"].astype(np.int32).values
cols = tmp["item_idx"].astype(np.int32).values

# Dimensiones:
n_users = tmp["user_idx"].nunique()
n_items = tmp["item_idx"].nunique()

# Construcción CSR:
R = csr_matrix((data, (rows, cols)), shape=(n_users, n_items))

print("Matriz CSR construida:")
print(" - shape (n_users, n_items):", R.shape)
print(" - nnz (no ceros / interacciones):", R.nnz)
print(" - densidad aproximada:", R.nnz / (R.shape[0] * R.shape[1]))

# Chequeo rápido de coherencia:
print("\nChequeos de consistencia:")
print(" - ¿Hay valores distintos de 1?:", np.any(R.data != 1.0))
print(" - min(data):", float(R.data.min()) if R.nnz > 0 else None)
print(" - max(data):", float(R.data.max()) if R.nnz > 0 else None)

Matriz CSR construida:
 - shape (n_users, n_items): (455586, 91187)
 - nnz (no ceros / interacciones): 494780
 - densidad aproximada: 1.1909919839927541e-05

Chequeos de consistencia:
 - ¿Hay valores distintos de 1?: True
 - min(data): 1.0
 - max(data): 10.0


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado:</b> la matriz usuario–ítem en formato CSR se ha construido correctamente a partir de las interacciones positivas indexadas, dando lugar a una representación altamente dispersa y escalable. Esta estructura permite almacenar de forma eficiente el gran volumen de usuarios e ítems del dataset y constituye la base sobre la que se implementará el componente colaborativo del sistema híbrido, manteniendo al mismo tiempo la flexibilidad necesaria para su integración con el componente basado en contenido.
</div>

<div style="background:#fff8e1;border-left:4px solid #f59e0b;padding:10px 12px;border-radius:6px;">
<b>Nota metodológica:</b> los valores almacenados en la matriz CSR representan la señal de interacción positiva previa a la aplicación de filtros colaborativos más estrictos. En fases posteriores, el componente Item-to-Item aplicará restricciones adicionales sobre esta matriz para garantizar estabilidad en el cálculo de similitudes, mientras que el componente basado en contenido operará sin requerir dichas restricciones.
</div>

<hr>

<h1>5. Diseño e implementación del sistema de recomendación híbrido</h1>

<p>
En este capítulo se describe el diseño y la implementación del sistema de recomendación híbrido,
que combina un enfoque de <b>Filtrado Colaborativo Item-to-Item</b> con un
<b>sistema de recomendación basado en contenido</b>.
</p>

<p>
Ambos componentes han sido desarrollados y analizados de forma independiente en capítulos anteriores.
El objetivo principal de este capítulo es <b>integrar ambas señales de recomendación</b> de manera coherente,
aprovechando sus fortalezas complementarias y mitigando sus limitaciones individuales.
</p>

<p>
El sistema híbrido permite:
</p>

<ul>
  <li>Generar recomendaciones incluso en escenarios de escasez de interacciones colaborativas.</li>
  <li>Mejorar la diversidad y relevancia de las recomendaciones.</li>
  <li>Reducir el impacto del problema de <i>cold-start</i>, especialmente para ítems nuevos.</li>
</ul>

<p>
En primer lugar, se define la <b>estrategia de hibridación</b> empleada, seguida de la implementación de
cada componente y de su combinación final para la generación de recomendaciones Top-N.
</p>

<h2>5.1. Estrategia de hibridación</h2>

<p>
El sistema de recomendación híbrido se construye siguiendo una estrategia de
<b>hibridación tardía (<i>late fusion</i>)</b>.
En este enfoque, cada componente del sistema genera recomendaciones de manera independiente,
que posteriormente se combinan en una fase de agregación.
</p>

<p>
En concreto, el sistema integra:
</p>

<ul>
  <li>
    <b>Señal colaborativa:</b> basada en Filtrado Colaborativo Item-to-Item,
    que explota patrones de co-ocurrencia en las interacciones usuario–ítem.
  </li>
  <li>
    <b>Señal de contenido:</b> basada en la similitud semántica entre ítems,
    calculada a partir de sus metadatos textuales.
  </li>
</ul>

<p>
La hibridación tardía presenta varias ventajas en este contexto:
</p>

<ul>
  <li>Permite reutilizar modelos ya entrenados de forma independiente.</li>
  <li>Facilita el análisis individual del impacto de cada señal.</li>
  <li>Ofrece flexibilidad para ajustar pesos y estrategias de combinación.</li>
</ul>

<p>
Las recomendaciones finales se obtienen mediante una combinación ponderada de los rankings
producidos por cada componente, asegurando que ambos contribuyen al resultado final
de forma controlada.
</p>

In [15]:
# ============================================================
# Estrategia de hibridación (late fusion)
# ============================================================

# Pesos de cada componente (configurables):
ALPHA_COLLAB = 0.6    # peso del filtrado colaborativo
ALPHA_CONTENT = 0.4  # peso del sistema basado en contenido

# Comprobación de coherencia:
assert abs(ALPHA_COLLAB + ALPHA_CONTENT - 1.0) < 1e-6, \
    "Los pesos de la hibridación deben sumar 1."

# Mostramos resultados obtenidos:
print("Estrategia de hibridación definida:")
print(f" - Peso colaborativo (Item-to-Item): {ALPHA_COLLAB}")
print(f" - Peso contenido (Content-Based):   {ALPHA_CONTENT}")

Estrategia de hibridación definida:
 - Peso colaborativo (Item-to-Item): 0.6
 - Peso contenido (Content-Based):   0.4


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado:</b><br>

En este apartado se ha definido formalmente la estrategia de hibridación del sistema de recomendación,
optando por un enfoque de <i>hibridación tardía (late fusion)</i>. Este enfoque permite integrar de forma
modular e interpretable las señales colaborativa y de contenido, manteniendo la independencia de ambos
componentes durante la generación de recomendaciones.
<br>

La asignación de pesos diferenciados (<code>0.6</code> para el componente colaborativo y <code>0.4</code> para el
componente basado en contenido) refleja una preferencia inicial por la señal colaborativa cuando existe
suficiente evidencia histórica, sin renunciar a la capacidad del sistema basado en contenido para aportar
valor en escenarios de baja densidad de interacciones o <i>cold-start</i>.
<br>

La validación explícita de la coherencia de los pesos garantiza la estabilidad del proceso de combinación
y facilita futuras fases experimentales en las que dichos pesos podrán ajustarse de forma controlada.
Esta estrategia establece una base sólida y flexible para la integración efectiva de ambos enfoques en
las siguientes etapas del sistema de recomendación híbrido.
</div>

<h2>5.2. Implementación del componente colaborativo (Item-to-Item)</h2>

<p>
El componente colaborativo del sistema híbrido se basa en un enfoque de
<b>Filtrado Colaborativo Item-to-Item</b>, reutilizando el modelo desarrollado y validado
previamente en el <i>Notebook 1</i>. Este enfoque explota patrones de co-ocurrencia entre
ítems a partir de las interacciones usuario–ítem representadas en la matriz
<b>CSR</b>.
</p>

<p>
El objetivo de este apartado es integrar dicho modelo dentro del sistema híbrido
<b>sin modificar su lógica interna</b>, garantizando coherencia metodológica y permitiendo
analizar de forma aislada la contribución del componente colaborativo frente al
componente basado en contenido.
</p>

<p>
En concreto, este componente:
</p>

<ul>
  <li>
    Utiliza la matriz usuario–ítem construida en el <b>Capítulo 4</b>.
  </li>
  <li>
    Calcula similitudes entre ítems a partir de interacciones implícitas.
  </li>
  <li>
    Genera recomendaciones de ítems similares a aquellos previamente consumidos por el usuario.
  </li>
</ul>

<p>
Las recomendaciones producidas por este módulo se combinarán posteriormente con las generadas
por el sistema basado en contenido mediante una <b>estrategia de hibridación tardía (late fusion)</b>,
lo que permite ajustar de forma flexible la influencia relativa de cada señal en el ranking final.
</p>

In [16]:
# ============================================================
# Componente colaborativo: Item-to-Item
# ============================================================

from sklearn.metrics.pairwise import cosine_similarity

# ------------------------------------------------------------
# Construimos matriz ítem-usuario a partir de R (CSR usuario x ítem)
# ------------------------------------------------------------
R_item_user = R.T.tocsr()

print("Matriz ítem-usuario construida:")
print(" - shape (n_items, n_users):", R_item_user.shape)
print(" - nnz:", R_item_user.nnz)

Matriz ítem-usuario construida:
 - shape (n_items, n_users): (91187, 455586)
 - nnz: 494780


In [17]:
# ------------------------------------------------------------
# Cálculo de similitud entre ítems (coseno)
# ------------------------------------------------------------
item_similarity = cosine_similarity(R_item_user, dense_output=False)

print("\nMatriz de similitud ítem-ítem:")
print(" - shape:", item_similarity.shape)
print(" - nnz:", item_similarity.nnz)


Matriz de similitud ítem-ítem:
 - shape: (91187, 91187)
 - nnz: 345287


In [18]:
# ------------------------------------------------------------
# Función de recomendación Item-to-Item
# ------------------------------------------------------------
def recommend_item_to_item(
    user_idx,
    R,
    item_similarity,
    top_n=10
):
    """
    Recomienda ítems a un usuario a partir del modelo Item-to-Item.
    """
    # Ítems con los que el usuario ha interactuado
    user_interactions = R[user_idx].nonzero()[1]

    # Score acumulado por ítem
    scores = np.zeros(R.shape[1], dtype=np.float32)

    for it in user_interactions:
        scores += item_similarity[it].toarray().ravel()

    # Eliminamos ítems ya consumidos
    scores[user_interactions] = 0.0

    # Top-N
    recommended_idx = np.argsort(scores)[::-1][:top_n]
    return recommended_idx, scores[recommended_idx]

In [19]:
# ------------------------------------------------------------
# Ejemplo de prueba con un usuario aleatorio
# ------------------------------------------------------------
sample_user_idx = np.random.randint(0, R.shape[0])

rec_items_idx, rec_scores = recommend_item_to_item(
    user_idx=sample_user_idx,
    R=R,
    item_similarity=item_similarity,
    top_n=10
)

print("\nEjemplo de recomendación Item-to-Item:")
print("Usuario índice:", sample_user_idx)
print("Ítems recomendados (idx):", rec_items_idx)
print("Scores:", rec_scores)


Ejemplo de recomendación Item-to-Item:
Usuario índice: 59790
Ítems recomendados (idx): [ 6586 30389 30390 30391 30392 30393 30394 30395 30396 30397]
Scores: [0.08164966 0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]


<div style="background:#f6f8fa;border-left:4px solid #1f6feb;padding:10px 12px;border-radius:6px;">
<b>Conclusión del apartado:</b><br>
El componente colaborativo Item-to-Item ha sido integrado correctamente dentro del sistema de recomendación híbrido, reutilizando de forma íntegra la lógica y estructura del modelo desarrollado en el Notebook 1. A partir de la matriz CSR usuario–ítem, se ha construido la representación ítem–usuario y se ha calculado una matriz de similitud ítem–ítem basada en la similitud del coseno, adecuada para datasets grandes y altamente dispersos.<br>
Los resultados obtenidos confirman que el modelo es capaz de generar recomendaciones Top-N coherentes para usuarios individuales, incluso en escenarios de historiales de interacción muy cortos, lo que refuerza su idoneidad para sistemas reales. Este módulo aporta una señal colaborativa sólida basada en patrones de co-ocurrencia globales y queda preparado para ser combinado, mediante hibridación tardía, con el componente basado en contenido en los siguientes apartados.
</div>